In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [36]:
jobs = pd.read_csv('ds_med_list.csv')
jobs.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Data Scientist,$70K-$116K (Glassdoor est.),"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech\n4.2,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1
1,Data Scientist,$70K-$116K (Glassdoor est.),"VA Boston Healthcare System\nBoston, Massachus...",3.6,VA Boston Healthcare System\n3.6,"Boston, MA","Boston, MA",201 to 500 employees,-1,Hospital,Health Care Services & Hospitals,Health Care,Unknown / Non-Applicable,-1
2,Data & Analytics Consultant,$70K-$116K (Glassdoor est.),WHO WE LOOK FOR\n\nAn SEI Consultant is a mast...,4.6,SEI\n4.6,"Cincinnati, OH","Cincinnati, OH",201 to 500 employees,1992,Company - Private,Consulting,Business Services,$50 to $100 million (USD),-1
3,R&D Scientist,$70K-$116K (Glassdoor est.),"Since 2001, Hygiena has been on a mission to b...",3.9,Hygiena\n3.9,"New Castle, DE","Camarillo, CA",201 to 500 employees,-1,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$100 to $500 million (USD),"Ecolab, Eurofin Private Investment Office"
4,Data Scientist,$70K-$116K (Glassdoor est.),Why We Work at Dun & Bradstreet\n\n\nWe are at...,3.1,Dun & Bradstreet\n3.1,"Austin, TX","Short Hills, NJ",1001 to 5000 employees,1841,Company - Private,IT Services,Information Technology,$1 to $2 billion (USD),"Experian, Equifax"


In [86]:
# cleaning the data using lambda functions can also uuse regular expressions

jobs = jobs[jobs['Salary Estimate'] != -1 ]
salary = jobs['Salary Estimate'].apply(lambda x: x.split('(')[0])
salary = salary.apply(lambda x: x.replace('K', '').replace('$',''))

jobs['Salary Estimate'] = salary
jobs['hourly'] = jobs['Salary Estimate'].apply(lambda x: 1 if 'per hour' in x.lower() else 0)
jobs['min_salary'] = jobs['Salary Estimate'].apply(lambda x: int(x.split('-')[0]))
jobs['max_salary'] = jobs['Salary Estimate'].apply(lambda x: int(x.split('-')[1]))
jobs['avg_salary'] =(jobs.min_salary + jobs.max_salary)/2
jobs['city'] = jobs['Location'].str.split(',', expand=True)[0]
jobs['state'] = jobs['Location'].str.split(',', expand=True)[1]
jobs['Company Name'] = jobs['Company Name'].apply(lambda x: x.split('\n')[0])
jobs['same_state'] = jobs.apply(lambda x: 1 if x.Location == x.Headquarters else 0, axis=1)
jobs['company_age'] = jobs['Founded'].apply(lambda x: x if x<1 else 2020-x)
jobs['num_competitors'] = jobs['Competitors'].apply(lambda x: len(x.split(',')) if x !='-1' else 0)

jobs.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,...,min_salary,max_salary,avg_salary,city,state,same_state,company_age,job_tag,job_level,num_competitors
0,Data Scientist,70-116,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,...,70,116,93.0,Chantilly,VA,0,52,data scientist,n/a,0
1,Data Scientist,70-116,"VA Boston Healthcare System\nBoston, Massachus...",3.6,VA Boston Healthcare System,"Boston, MA","Boston, MA",201 to 500 employees,-1,Hospital,...,70,116,93.0,Boston,MA,1,-1,data scientist,n/a,0
2,Data & Analytics Consultant,70-116,WHO WE LOOK FOR\n\nAn SEI Consultant is a mast...,4.6,SEI,"Cincinnati, OH","Cincinnati, OH",201 to 500 employees,1992,Company - Private,...,70,116,93.0,Cincinnati,OH,1,28,other,n/a,0
3,R&D Scientist,70-116,"Since 2001, Hygiena has been on a mission to b...",3.9,Hygiena,"New Castle, DE","Camarillo, CA",201 to 500 employees,-1,Company - Private,...,70,116,93.0,New Castle,DE,0,-1,other,n/a,2
4,Data Scientist,70-116,Why We Work at Dun & Bradstreet\n\n\nWe are at...,3.1,Dun & Bradstreet,"Austin, TX","Short Hills, NJ",1001 to 5000 employees,1841,Company - Private,...,70,116,93.0,Austin,TX,0,179,data scientist,n/a,2


In [72]:
# creating tags for keywords in job titles

def title_keywords(title):
    if 'data scientist' in title.lower():
        return 'data scientist'
    elif 'engineer' in title.lower():
        return 'data engineer'
    elif 'analyst' in title.lower():
        return 'data analyst'
    elif 'machine learning' in title.lower():
        return 'mle'
    else:
        return 'other'
    
def job_level(title):
    if 'sr' in title.lower() or 'senior' in title.lower() or 'experienced' in title.lower() or 'lead' in title.lower() or 'principal' in title.lower():
        return 'Experienced'
    elif 'jr' in title.lower() or 'junior' in title.lower() or 'associate' in title.lower() or 'entry' in title.lower() or 'early' in title.lower():
        return 'Associate/Jr. Level'
    else:
        return 'n/a'

In [73]:
jobs['job_tag'] = jobs['Job Title'].apply(title_keywords)
jobs['job_level'] = jobs['Job Title'].apply(job_level)

In [74]:
jobs.head(15)

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,...,hourly,min_salary,max_salary,avg_salary,city,state,same_state,company_age,job_tag,job_level
0,Data Scientist,70-116,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,...,0,70,116,93.0,Chantilly,VA,0,52,data scientist,n/a
1,Data Scientist,70-116,"VA Boston Healthcare System\nBoston, Massachus...",3.6,VA Boston Healthcare System,"Boston, MA","Boston, MA",201 to 500 employees,-1,Hospital,...,0,70,116,93.0,Boston,MA,1,-1,data scientist,n/a
2,Data & Analytics Consultant,70-116,WHO WE LOOK FOR\n\nAn SEI Consultant is a mast...,4.6,SEI,"Cincinnati, OH","Cincinnati, OH",201 to 500 employees,1992,Company - Private,...,0,70,116,93.0,Cincinnati,OH,1,28,other,n/a
3,R&D Scientist,70-116,"Since 2001, Hygiena has been on a mission to b...",3.9,Hygiena,"New Castle, DE","Camarillo, CA",201 to 500 employees,-1,Company - Private,...,0,70,116,93.0,New Castle,DE,0,-1,other,n/a
4,Data Scientist,70-116,Why We Work at Dun & Bradstreet\n\n\nWe are at...,3.1,Dun & Bradstreet,"Austin, TX","Short Hills, NJ",1001 to 5000 employees,1841,Company - Private,...,0,70,116,93.0,Austin,TX,0,179,data scientist,n/a
5,"Scientist, Pharmacology",70-116,Sutro has a unique capability that enables new...,4.3,Dun & Bradstreet,"Austin, TX","South San Francisco, CA",51 to 200 employees,2003,Company - Public,...,0,70,116,93.0,Austin,TX,0,17,other,n/a
6,Sr. Data Scientist,70-116,Sr. Data Scientist\nCurative is scaling up COV...,4.0,"Curative Inc,","San Dimas, CA","Racine, WI",51 to 200 employees,1949,Nonprofit Organization,...,0,70,116,93.0,San Dimas,CA,0,71,data scientist,Experienced
7,Data Scientist,70-116,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,...,0,70,116,93.0,Newton,MA,0,20,data scientist,n/a
8,Data Scientist,70-116,The Data Scientist may be responsible for any ...,2.8,Sandhills Global,"Lincoln, NE","Lincoln, NE",1001 to 5000 employees,1978,Company - Private,...,0,70,116,93.0,Lincoln,NE,1,42,data scientist,n/a
9,Data Scientist,70-116,Company Overview\n\nImagine if we could match ...,2.9,GNS Healthcare,"Cambridge, MA","Cambridge, MA",51 to 200 employees,2000,Company - Private,...,0,70,116,93.0,Cambridge,MA,1,20,data scientist,n/a


In [90]:
jobs.to_csv('cleaned_list.csv', index=False)